# Looking for the objects in the catalogs

We will try to find the onjects from the lst in the catalogs TeV, GeV and Simbad.

- PSR B1259-63
- LS 5039
- LS   I +61  132
- HESS J0632+057
- PSR J2032+4127
- 1FGL J1018.6-5856
- HESS J0536-675


In [1]:
import pandas as pd
import re
import sys
from definitions import *

In [2]:
regular_expressions = ['.*B12.*63.*',
                       '.*LS.*5039.*',
                       '.*LS.*I.*61.*303.*',
                       '.*J0632.*057.*',
                       '.*J2032.*.*4127.*',
                       '.*1FGL.*J1018.*5856.*',
                       '.*J0536.*67.*',
                      ]

This cell loads data created in simbad.ipynb

In [3]:
cat_name1 = get_name_for_gevtevxmm()
cat_name2 = get_name_for_gevtev()
tevsimbad = pd.read_csv("data/tevsimbad.txt", sep='\t', index_col=0)
gevsimbad = pd.read_csv("data/gevsimbad.txt", sep='\t', index_col=0)
gevtevsimbad = pd.read_csv("data/gevtevsimbad.txt", sep='\t', index_col=0)
pd_gev = pd.read_csv("data/gev.txt", sep='\t', index_col=0)
pd_tev = pd.read_csv("data/tev.txt", sep='\t', index_col=0)
pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd.read_csv("data/{}.txt".format(cat_name2), sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.loc[:, ~pd_gevtev_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.reset_index(drop=True)
pd_gevtev_simbadclasses.to_csv("data/{}.txt".format(cat_name2), sep='\t')
pd_gevtevxmm_simbadclasses = pd.read_csv("data/{}.txt".format(cat_name1), sep = '\t', index_col=0)
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.loc[:, ~pd_gevtevxmm_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.reset_index(drop=True)
pd_gevtevxmm_simbadclasses.to_csv("data/{}.txt".format(cat_name1), sep='\t')

In [4]:
print(len(pd_gevtev_simbadclasses))
print(len(pd_gevtevxmm_simbadclasses))

141
2175


The following cell will be useful to determine a string name of a catalog where an object is found. It is not the cleverest way to do it, as if number of elements is changed(another epsilon is used), this dictionary should be corrected manually.

In [5]:
len_catalogs = {len(tevsimbad) : 'tevsimbad',
            len(gevsimbad) :'gevsimbad',
            len(gevtevsimbad) :'gevtevsimbad',
            len(pd_gev) : 'gev',
            len(pd_tev) : 'tev',
            len(pd_simbad) : 'simbad',
            len(pd_gevtev_simbadclasses) : cat_name2,
            len(pd_gevtevxmm_simbadclasses) : cat_name1
               }

In [6]:
name_to_catalog = {'tevsimbad' : tevsimbad,
            'gevsimbad' : gevsimbad,
            'gevtevsimbad' : gevtevsimbad,
            'gev' : pd_gev,
            'tev' : pd_tev,
            'simbad' : pd_simbad,
            cat_name2 : pd_gevtev_simbadclasses,
            cat_name1 : pd_gevtevxmm_simbadclasses}

In [7]:
gev_columns = ['gev_2FGL_Name', 'gev_1FGL_Name', 
                 'gev_1FHL_Name', 'gev_2FGL_Name', 
                 'gev_ASSOC1', 'gev_ASSOC2',
                'gev_ASSOC_GAM1', 'gev_ASSOC_GAM2',
                'gev_ASSOC_GAM3', 'gev_ASSOC_TEV', 'gev_CLASS1',
                'gev_GLAT', 'gev_GLON']

tev_columns = ['tev_common_name', 'tev_gamma_names', 
                'tev_fermi_names', 'tev_other_names', 
                'tev_classes',
                'tev_glat', 'tev_glon']

s_columns = ['s_class', 's_spectra', 's_glat', 's_glon',
            's_MAIN_ID']

xmm_columns = ['xmm_IAUNAME']

s_tev_columns = tev_columns + s_columns
s_gev_columns = gev_columns + s_columns
tev_gev_columns = gev_columns + tev_columns
tev_gev_columns_s = gev_columns + tev_columns + ['s_class', 's_simbad', 's_MAIN_ID']
gevtevs_columns = gev_columns + tev_columns + s_columns
tev_gev_xmm_columns_s = gev_columns + tev_columns + xmm_columns + ['s_class', 's_simbad', 's_MAIN_ID']

In [8]:
name_to_column = {'tevsimbad' : s_tev_columns,
            'gevsimbad' : s_gev_columns,
            'gevtevsimbad' : gevtevs_columns,
            'gev' : gev_columns,
            'tev' : tev_columns,
            'simbad' : s_columns,
            cat_name2 : tev_gev_columns_s,
            cat_name1 : tev_gev_xmm_columns_s}

In [9]:
def find_in_catalogs(string_to_find, catalog):
    """Looks for an object containing in its name string_to_find in the catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    """
    for c in catalog.columns:
        for obj in catalog[c]:
            if (str(obj)).find(string_to_find) != -1:
                print(len_catalogs[len(catalog)] + ' :')
                print('\t' + str(c))
                print('\t\t' + (str(obj)))
            


In [10]:
def old_find_number_in_catalogs(string_to_find, catalog, printRes = False):
    #Based on string find
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):

            obj = catalog[c][number_obj]
            if (str(obj)).find(string_to_find) != -1:
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [11]:
def find_number_in_catalogs(regular_expression, catalog, printRes = False):
    #based on regular expressions
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):
            try:
                obj = catalog[c][number_obj]
            except:
                print(c)
                display(catalog[c])
                print(number_obj)
            if (re.search(regular_expression, str(obj)) is not None):
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [12]:
catalogs = [#gevtevsimbad,
            #tevsimbad,
            #gevsimbad,
            pd_gevtev_simbadclasses,
            pd_gevtevxmm_simbadclasses
           ]

In [13]:
separate_catalogs = [
    pd_gev,
    pd_tev,
    pd_simbad,
]

Strings we want to find in the catalogs:

In [14]:
strings_to_find = [
    'PSR B1259-63',
    'LS 5039',
    'LS   I +61  303',
    'HESS J0632+057',
    'LAT PSR J2032+4127',
    '1FGL J1018.6-5856',
    'J0536',
    
]

In [15]:
resultats_of_search = {}
#this dictionary remembers catalog name and corresponding number of line where an interesting objects is found
for i in len_catalogs.values():
    resultats_of_search.update({i : []})

In [16]:
for regular_expression in regular_expressions:
    print('Looking for : ' + regular_expression)
    for j in catalogs:

        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
        print(len_catalogs[len(j)])
        if success:
            resultats_of_search[catalog_name].append(number_obj)
print('Done')

Looking for : .*B12.*63.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*LS.*5039.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*LS.*I.*61.*303.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*J0632.*057.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*J2032.*.*4127.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*1FGL.*J1018.*5856.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Looking for : .*J0536.*67.*
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
Done


In [17]:
print(resultats_of_search)

{'tevsimbad': [], 'gevsimbad': [], 'gevtevsimbad': [], 'gev': [], 'tev': [], 'simbad': [], 'gevtev_simbadclasses_25': [112, 12, 131, 44], 'gevtevxmm_simbadclasses_06': [1745, 239, 1980, 618]}


Looking in separate catalogs:

In [18]:
#for regular_expression in regular_expressions:
#    print('Looking for : ' + regular_expression)
#    for j in separate_catalogs:
#        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
#        if success:
#            resultats_of_search[catalog_name].append(number_obj)
#print('Done')

In [19]:
print(resultats_of_search)

{'tevsimbad': [], 'gevsimbad': [], 'gevtevsimbad': [], 'gev': [], 'tev': [], 'simbad': [], 'gevtev_simbadclasses_25': [112, 12, 131, 44], 'gevtevxmm_simbadclasses_06': [1745, 239, 1980, 618]}


In [20]:
pd_gevtevxmm_simbadclasses 

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
0,6450,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.496947,0.001304,-0.844130,0.001332,0.531463,6.207905,4,84,200962101010013,b'http://xmm-catalog.irap.omp.eu/source/200962...
1,6451,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.508346,0.001276,-0.898199,0.001489,0.535606,6.208424,4,84,200962101010004,b'http://xmm-catalog.irap.omp.eu/source/200962...
2,6458,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.711103,0.033164,-1.000000,0.011269,1.515710,6.215158,3,70,204123802010008,b'http://xmm-catalog.irap.omp.eu/source/204123...
3,6462,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.574087,0.004744,-0.995505,0.002180,0.568375,6.216956,3,84,200962101010003,b'http://xmm-catalog.irap.omp.eu/source/200962...
4,6478,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.516946,0.001762,-0.907838,0.001398,1.060870,6.244211,4,84,200962101010002,b'http://xmm-catalog.irap.omp.eu/source/200962...
5,6481,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.578960,0.009740,-0.998252,0.003187,0.868645,6.251406,3,84,204123801010001,b'http://xmm-catalog.irap.omp.eu/source/204123...
6,6482,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.515595,0.000796,-0.910853,0.000869,0.567426,6.251975,4,84,203105901010001,b'http://xmm-catalog.irap.omp.eu/source/203105...
7,6487,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.387785,0.020930,-0.978735,0.010924,1.509070,6.256357,3,70,203105901010006,b'http://xmm-catalog.irap.omp.eu/source/203105...
8,6492,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.527928,0.064098,-0.974951,0.038941,1.513510,6.260420,3,84,200962101010001,b'http://xmm-catalog.irap.omp.eu/source/200962...
9,6493,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.417619,0.023096,-0.994708,0.007770,1.516650,6.262618,3,84,200962101010006,b'http://xmm-catalog.irap.omp.eu/source/200962...


In [21]:
pd_gevtevxmm_simbadclasses.iloc[resultats_of_search[cat_name1]]

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
1745,412166,b'0FGL J1826.3-1451',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',...,0.380675,0.578834,0.650810,0.186760,1.84189,280.178478,0,0,206048203010108,b'http://xmm-catalog.irap.omp.eu/source/206048...
239,69007,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,-1.000000,0.621338,1.000000,0.583818,2.28100,40.022122,0,0,205059811010050,b'http://xmm-catalog.irap.omp.eu/source/205059...
1980,438827,b' ',b'1FGL J2015.7+3708 ',b'1FHL J2015.8+3710 ',b'2FGL J2015.6+3709 ',b'MG2 J201534+3710 ',b' ',b' ',b'3EG J2016+3657',b' ',...,0.081720,0.165468,-0.369797,0.227730,1.32049,308.007282,3,0,203055602010047,b'http://xmm-catalog.irap.omp.eu/source/203055...
618,212176,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,-1.000000,0.024570,NaN,NaN,2.84193,154.632681,3,0,206943901010027,b'http://xmm-catalog.irap.omp.eu/source/206943...


In [22]:
pd_gevtev_simbadclasses.iloc[resultats_of_search[cat_name2]]

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
112,2358,b'0FGL J1826.3-1451',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',...,b'ecpl',b'xKaTtF',96,b'TeV J1826-148',119,b'TeV J1826-1449',b'gal',"[""b'LS IV -14 55'"" ""b'BD-14 5047'"" ""b'V* V...",['B' 'B0Ve' 'B0Ve'],1
12,319,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,b'pl',b'fW749W',89,b'TeV J0240+612',14,b'TeV J0240+6115',b'gal',"[""b'BD+60 529'"" ""b'EM* MWC 56'"" ""b'LS I ...",['B' 'B' 'B0Ve' 'B0Ve'],1
131,2648,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,b'none',b'1FCN0q',149,b'TeV J2031+406',145,b'TeV J2031+4040',b'gal',"[""b'[D75b] Em* 20+118'"" ""b'[D75b] Em* 20+118'""...",['Be' 'B' 'B0Ve' 'B0Ve' 'B0Ve'],1
44,1172,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,b'pl',b'5FlzLA',237,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',"[""b'2FGL J1019.0-5856'""]",['B0Ve'],2


In [23]:
for i in resultats_of_search.keys():
    if i.find('simbad') != -1 and i.find('simbadclasses') == -1:
        continue
    print(i + ' :')
    display(name_to_catalog[i][name_to_column[i]].loc[resultats_of_search[i]])

gev :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,gev_CLASS1,gev_GLAT,gev_GLON


tev :


,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon


gevtev_simbadclasses_25 :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,s_class,s_simbad,s_MAIN_ID
112,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'LS 5039',b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,['B' 'B0Ve' 'B0Ve'],1,"[""b'LS IV -14 55'"" ""b'BD-14 5047'"" ""b'V* V..."
12,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'LS I +61 303',b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,['B' 'B' 'B0Ve' 'B0Ve'],1,"[""b'BD+60 529'"" ""b'EM* MWC 56'"" ""b'LS I ..."
131,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',...,b'MGRO J2031+41',b'',b'',"b'TeV J2032+4130,TeV 2032+413,0FGL J2032.2+4122'",b'unid',1.111117,80.299477,['Be' 'B' 'B0Ve' 'B0Ve' 'B0Ve'],1,"[""b'[D75b] Em* 20+118'"" ""b'[D75b] Em* 20+118'""..."
44,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 A',b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,['B0Ve'],2,"[""b'2FGL J1019.0-5856'""]"


gevtevxmm_simbadclasses_06 :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,xmm_IAUNAME,s_class,s_simbad,s_MAIN_ID
1745,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,b'3XMM J184042.8-053159',['B' 'B0Ve' 'B0Ve'],1,"[""b'LS IV -14 55'"" ""b'BD-14 5047'"" ""b'V* V..."
239,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,b'3XMM J024005.3+611247',['B' 'B' 'B0Ve' 'B0Ve'],1,"[""b'BD+60 529'"" ""b'EM* MWC 56'"" ""b'LS I ..."
1980,b'2FGL J2015.6+3709 ',b'1FGL J2015.7+3708 ',b'1FHL J2015.8+3710 ',b'2FGL J2015.6+3709 ',b'MG2 J201534+3710 ',b' ',b' ',b'3EG J2016+3657',b' ',b'VER J2016+371 ',...,b'MGRO J2019+37',b'',b'CTB 87',"b'pwn,snr'",1.150181,74.953850,b'3XMM J203201.7+412709',['Be' 'B'],1,"[""b'Hen 2-461'"" ""b'Hen 2-461'""]"
618,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,b'3XMM J101831.8-585524',['B0Ve'],2,"[""b'2FGL J1019.0-5856'""]"


The following cell saves information about objects found.

In [24]:
#gevtevsimbad[gevtevs_columns].to_csv('resultat_gevtevsimbad.txt',sep = '\t')
#gevtev[tev_gev_columns].to_csv('resultat_gevtev.txt',sep = '\t')
#gevsimbad[s_gev_columns].to_csv('resultat_sgev.txt',sep = '\t')
#tevsimbad[s_tev_columns].to_csv('resultat_stev.txt',sep = '\t')

In [25]:
#for i in resultats_of_search.keys():
#    (name_to_catalog[i].loc[resultats_of_search[i]]).to_csv('resultats/resultat_' + i +'.txt',sep = '\t')

The following cell adds to bases used in the beginning a column that indicates the objects choosen in this ipynb.

In [26]:
for i in resultats_of_search.keys():
    print(i)
    if i in ['gev','tev','simbad']:
        continue
    catalog = name_to_catalog[i].copy()
    catalog.is_copy = False
    catalog['marked'] = 0
    (catalog.loc[resultats_of_search[i], 'marked'])= 1
    (catalog).to_csv('markeddata/' + i +'.txt',sep = '\t')


tevsimbad
gevsimbad


/home/masha/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/home/masha/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


gevtevsimbad
gev
tev
simbad
gevtev_simbadclasses_25
gevtevxmm_simbadclasses_06
